# Pokkelpop

# 1. Setup and Imports
Import the necessary libraries for web scraping and data handling.

In [36]:
import json  
# Import the JSON library to handle JSON data for reading from or writing to JSON files.

from selenium.webdriver.chrome.service import Service  
# Import the Service class to manage the ChromeDriver service for Selenium.

from webdriver_manager.chrome import ChromeDriverManager  
# Import the ChromeDriverManager to automatically download and manage the correct version of ChromeDriver.

from selenium import webdriver  
# Import the main WebDriver class to control the Chrome browser.

from selenium.webdriver.common.by import By  
# Import the By class for locating elements on a webpage using different strategies (e.g., ID, class, CSS selector).

from selenium.webdriver.support.ui import WebDriverWait  
# Import the WebDriverWait class to wait for certain conditions to be met before proceeding with actions.

from selenium.webdriver.support import expected_conditions as EC  
# Import the `expected_conditions` module to define specific conditions (e.g., element presence) for WebDriverWait.

from selenium.common.exceptions import TimeoutException, NoSuchElementException  
# Import exceptions to handle errors like timeouts or missing elements during web scraping.

from bs4 import BeautifulSoup  
# Import BeautifulSoup for parsing and navigating HTML documents efficiently.

import re  
# Import the regular expressions library for text pattern matching and manipulation.


# 2. Generate Festival URLs
Create a list of URLs for festival years from 2024 to 2000.

In [37]:
# List to hold all URLs
url_list = []

# Generate URLs from 2024 down to 2000
for year in range(2024, 1984, -1):
    url_list.append(f"https://www.pukkelpop.be/nl/geschiedenis/{year}")

# Print the generated URLs
print("Generated URLs:")
for url in url_list:
    print(url)


Generated URLs:
https://www.pukkelpop.be/nl/geschiedenis/2024
https://www.pukkelpop.be/nl/geschiedenis/2023
https://www.pukkelpop.be/nl/geschiedenis/2022
https://www.pukkelpop.be/nl/geschiedenis/2021
https://www.pukkelpop.be/nl/geschiedenis/2020
https://www.pukkelpop.be/nl/geschiedenis/2019
https://www.pukkelpop.be/nl/geschiedenis/2018
https://www.pukkelpop.be/nl/geschiedenis/2017
https://www.pukkelpop.be/nl/geschiedenis/2016
https://www.pukkelpop.be/nl/geschiedenis/2015
https://www.pukkelpop.be/nl/geschiedenis/2014
https://www.pukkelpop.be/nl/geschiedenis/2013
https://www.pukkelpop.be/nl/geschiedenis/2012
https://www.pukkelpop.be/nl/geschiedenis/2011
https://www.pukkelpop.be/nl/geschiedenis/2010
https://www.pukkelpop.be/nl/geschiedenis/2009
https://www.pukkelpop.be/nl/geschiedenis/2008
https://www.pukkelpop.be/nl/geschiedenis/2007
https://www.pukkelpop.be/nl/geschiedenis/2006
https://www.pukkelpop.be/nl/geschiedenis/2005
https://www.pukkelpop.be/nl/geschiedenis/2004
https://www.pukkel

# 3. Initialize Selenium WebDriver
Set up Selenium to automate browser interactions.

In [38]:
# Set up the Selenium WebDriver
service = Service(ChromeDriverManager().install())  
# Initialize the ChromeDriver service using `ChromeDriverManager().install()`, which automatically installs and manages the appropriate version of ChromeDriver for Selenium.

options = webdriver.ChromeOptions()  
# Create an instance of `ChromeOptions` to set up browser-specific options, such as running in headless mode or customizing the browser behavior.

# Uncomment below for headless mode
# options.add_argument("--headless")  
# If you want to run the browser in headless mode (without a visible window), uncomment this line.
# Headless mode is useful for running scripts on a server or in an environment where you don’t need a GUI.

driver = webdriver.Chrome(service=service, options=options)  
# Create an instance of `webdriver.Chrome`, passing in the service and options.
# This initializes the Chrome browser and begins automation with the specified configurations.


# 4. Scrape Data for Each Year
Fetch and parse data from each URL, storing results in a dictionary.

In [39]:
# Dictionary to hold data for each year
festival_data = {}  
# Initialize an empty dictionary to store artist data, where the key is the year and the value is the list of artists.

# Loop through each URL in url_list
for url in url_list:
    # Extract the year from the URL using a regular expression.
    year_match = re.search(r'(\d{4})$', url)  
    # The regular expression `(\d{4})$` captures the 4-digit year at the end of the URL.
    year = int(year_match.group(0)) if year_match else "Unknown Year"  
    # If the year is found, convert it to an integer; otherwise, label it as "Unknown Year".

    try:
        # Open the webpage using Selenium WebDriver.
        driver.get(url)
        
        # Wait for the page to load and for the list of acts (artists) to appear.
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.act-edition-detail__acts"))
            # Wait for the specific <ul> element with the class `act-edition-detail__acts` to be present on the page.
        )
        
        # Get the page source (HTML content) after JavaScript has loaded the necessary content.
        html = driver.page_source
        
        # Parse the HTML content using BeautifulSoup to extract the data.
        soup = BeautifulSoup(html, 'html.parser')
        
        # Find the <ul> element that contains the list of artists.
        acts_list = soup.find('ul', class_='act-edition-detail__acts')
        
        # Initialize an empty list to store artist names for this specific year.
        artist_list = []
        
        # Check if the target <ul> element was found on the page.
        if acts_list:
            # Loop through all <li> tags within the found <ul> and extract the artist name from each.
            for li_tag in acts_list.find_all('li'):
                artist_text = li_tag.get_text(strip=True)  # Extract the text, removing extra spaces.
                artist_list.append(artist_text)  # Add the artist name to the list.
            
            # If artist names were found, store them in the dictionary with the year as the key.
            if artist_list:
                festival_data[year] = artist_list
            else:
                # If no artists are found for the year, print a message indicating no data.
                print(f"There’s no data for the year {year}.")
        else:
            # If the target <ul> element wasn't found, print a message indicating no data for the year.
            print(f"There’s no data for the year {year}.")
    
    # Handle exceptions in case of timeout or missing elements.
    except (TimeoutException, NoSuchElementException):
        print(f"There’s no data for the year {year}.")
    # Handle any other unexpected exceptions and print the error message.
    except Exception as e:
        print(f"Error accessing data for {year}: {e}")


There’s no data for the year 2024.
There’s no data for the year 2021.
There’s no data for the year 2020.
There’s no data for the year 1989.


# 5. Close Selenium WebDriver
Shut down the WebDriver after scraping is completed.

In [40]:
# Close the Selenium WebDriver
driver.quit()


# 6. Print Scraped Data
Print the collected data in a readable format.

In [41]:
# Print the scraped data in the specified format
for year, artists in sorted(festival_data.items(), reverse=True):  
    # Loop through the `festival_data` dictionary, sorted by year in descending order (most recent year first).
    print(f"{year}:")  
    # Print the year as a heading for the list of artists.

    for artist in artists:  
        # Loop through the list of artists for the given year.
        print(f"  - {artist}")  
        # Print each artist with a dash (`-`) for formatting, to indicate each artist name.

    print()  
    # Print a blank line after the list of artists for each year to separate the entries for better readability.


2023:
  - Wannes Lenaers
  - Omdat Het Kan & Average Rob
  - Hindu Radio DJ's
  - Keyser
  - Kwijt Trek System
  - Olly Ameen
  - Boiler Boys
  - She The DJ
  - $hirak
  - Sophie Straat
  - DC Noises
  - Gomorris
  - freq444
  - blackwave.
  - PSYCHONAUT
  - Hyalyte
  - KRANKk
  - Black Box Revelation
  - Flo Windey & Skyve
  - Portland
  - Tsar B
  - Ax Emblem
  - The Subs + special guests
  - Andromedik
  - The Hickey Underworld
  - Blawan
  - Amelie Lens
  - Moderat
  - ascendant vierge
  - DJ Fresh
  - DJ St. Paul
  - Ilse Liebens & Nadiem Shah
  - Timmerman B2B Sara Dziri
  - Elmer
  - Siegfried & Joy
  - Black Leather Jacket
  - Clash Royale BESF Cup
  - Only The Poets
  - BLUAI
  - DIRK.
  - Zulu
  - sor
  - Royel Otis
  - Froukje
  - Ploegendienst
  - EMJIE
  - MEMORO by Joyhauser (live)
  - Nessa Barrett
  - Rolo Tomassi
  - Hannah Grae
  - Ava Eva
  - Helena Lauwaert
  - Ethel Cain
  - Bazart
  - Brutus
  - The Aces
  - ArrDee
  - Bob Vylan
  - Asian Sal
  - Jan Vercauteren
 

# 7. Save Data to a JSON File
Save the results into a JSON file for future use.

In [42]:
# Save the data to a JSON file
output_file = 'pokkelpop_festival.json'  
# Specify the name of the JSON output file where the data will be saved.

# Open the specified file in write mode with UTF-8 encoding to handle non-ASCII characters.
with open(output_file, 'w', encoding='utf-8') as json_file:  
    # Open the file `pokkelpop_festival.json` for writing (`'w'` mode).
    # The `encoding='utf-8'` ensures that any non-ASCII characters (like accented letters or special symbols) are correctly handled.

    # Use the json.dump function to write the festival data to the file in a readable format.
    json.dump(festival_data, json_file, ensure_ascii=False, indent=4)  
    # `festival_data`: The data to write to the JSON file.
    # `ensure_ascii=False`: Ensures non-ASCII characters are written correctly (e.g., characters with accents).
    # `indent=4`: Makes the output JSON file more human-readable by formatting it with 4 spaces of indentation.

# Print a success message indicating the data has been saved successfully.
print(f"Data successfully saved to {output_file}")  
# Output a confirmation message to the console indicating where the data has been saved.


Data successfully saved to pokkelpop_festival.json
